Computing bids for the hybrid model

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime

In [2]:
df = pd.read_csv("Root_Insurance_data.csv")

Get probabilities via logistic regression using features 'Currently Insured_Y', number of vehicles, and number of drivers as categorical variables.

In [22]:
from sklearn.linear_model import LogisticRegression

glm = LogisticRegression()
glm.fit(
    pd.get_dummies(df[["Currently Insured","Number of Vehicles","Number of Drivers"]],
        columns=["Currently Insured","Number of Vehicles","Number of Drivers"])\
    .loc[df.click, ["Currently Insured_Y","Number of Vehicles_2","Number of Vehicles_3","Number of Drivers_2"]].to_numpy(),
    df["policies_sold"].loc[df.click].to_numpy()
)

LogisticRegression()

Get list of customer probabilities to buy and parameters for competing bid distributions.

In [83]:
# number of categories
cat = 36
total = np.zeros(cat) # total number in each category
bid = 10*np.ones(cat) # default bids set to 10
probSoldGivenClick = np.zeros(cat)
# parameters for the hybrid distribution
_lambda = np.zeros(cat)
_p = np.zeros(cat)


theta = df.groupby("rank").click.mean().to_numpy() # get estimates for click rates by rank

# add column to track indexnumber
df["class"] = 0
idx = 0
ls = pd.DataFrame( columns = ["insured", "cars", "drivers","married"])
for i in ["Y","N","unknown"]:
    for c in [1,2,3]:
        for d in [1,2]:
            for m in ["M","S"]:
                probSoldGivenClick[idx] = glm.predict_proba(
                    np.array([[(i=="Y"),(c==2),(c==3),(d==2)]])
                    )[0,1]
                # record index number
                df.loc[(df["Currently Insured"]==i)&(df["Number of Vehicles"]==c)&\
                           (df["Number of Drivers"]==d)&(df["Marital Status"]==m), "class"] = idx
                total[idx] = df.loc[ (df["Currently Insured"]==i)&(df["Number of Vehicles"]==c)&\
                           (df["Number of Drivers"]==d)&(df["Marital Status"]==m) ].shape[0]
                r = df["rank"].loc[ df["class"]==idx ].mean()
                if np.isnan(r):
                    r = df["rank"].mean()
                _p[idx] = (r-1)/4
                _lambda[idx] = 10*(r-1)/(5-r)
                ls = ls.append({"insured":i, "cars":c, "drivers":d, "married":m}, ignore_index=True)
                idx+=1
                

In [84]:
#expectedcost, expectedsold, expected cost per policy sold
def expectedsold(bid):#expected number of policies sold
    for i in range(0, len(bid)):
        if bid[i]<0:
            return 0
    e = 0
    for i in range(0, 35):#35 because click at 36 = 0, lambda will cause errors
        for r in range(1, 6):
            if bid[i]<0:
                e = e +0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*\
                    (1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*\
                    (_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
    return e
def expectedcost(bid):#expected cost of policy
    e = 0
    for i in range(0, 35):
        for r in range(1, 6):
            if bid[i]<0:
                e = e+0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*\
                    (1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*\
                    (_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
        #print(e)
    return e

def costpersold(bid):
    return expectedcost(bid)/expectedsold(bid)
def constraint(bid): #constaining that the expected number of policies sold is more than 400
    return expectedsold(bid)


In [85]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint, NonlinearConstraint

In [86]:
#hessian is zero matrix if needed
def cons_H(x, v):
    return np.zeros((36,36))
#nonlinar constraint is the constraint function is bounded from 400 to 1000
nonlinear_constraint = NonlinearConstraint(constraint, 400, 1000)#, hess=cons_H)
#linear constraint is each bid is between 1 to 50
lincon = LinearConstraint(np.identity(36), np.linspace(0.01, 0.01, num=36), np.linspace(50000, 50000, num=36))

In [109]:
# get constrained optimization 
nonlin =[]
for i in range(3,5):
    nonlin.append(NonlinearConstraint(constraint, 106+i*100, 106+i*100, hess=cons_H))
    
res = []
print(datetime.datetime.now())
for i in range(2):
    res.append(minimize(expectedcost, bid, method='trust-constr', constraints=[lincon, nonlin[i]],  options={'maxiter':5000})) 
    print(datetime.datetime.now())
for i in range(2):
    print(expectedcost(res[i].x), expectedsold(res[i].x), costpersold(res[i].x))

2021-05-28 14:38:52.806317
2021-05-28 14:39:54.091288
2021-05-28 14:40:37.165379
5336.408090022858 406.00000000000017 13.14386229069669
8110.008548031037 506.0 16.027684877531694


In [110]:
ls["bid"] = res[0].x
ls.sort_values("bid",ascending=False)

,insured,cars,drivers,married,bid
35,unknown,3,2,S,9.934160
25,unknown,1,1,S,8.271301
24,unknown,1,1,M,8.225060
26,unknown,1,2,M,6.988407
27,unknown,1,2,S,6.973347
15,N,1,2,S,6.396107
13,N,1,1,S,6.386141
14,N,1,2,M,6.384859
29,unknown,2,1,S,6.291145
28,unknown,2,1,M,6.244354


In [111]:
niter = 10000
arr = np.zeros(niter)
for i in range(niter):
    batch = np.random.choice(df["class"], size=10000, replace=True)
    # need to modify totals 
    for j in range(cat):
        total[j] = np.count_nonzero(batch==j)
    arr[i] = expectedsold(res[0].x)
m = np.mean(arr)
s = np.std(arr)
print("Mean: ",m)
print("Standard Deviation: ",s)

Mean:  406.53012874252425
Standard Deviation:  3.8886284257150905


In [112]:
from scipy.stats import norm

norm.cdf(400, loc=m,scale=s)

0.04654792192755653

In [113]:
import statsmodels.api as sm

lm = sm.OLS(ls["bid"],sm.add_constant(pd.get_dummies(ls[["insured","cars","drivers","married"]]))).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    bid   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     30.50
Date:                Fri, 28 May 2021   Prob (F-statistic):           2.89e-11
Time:                        14:40:51   Log-Likelihood:                -48.050
No. Observations:                  36   AIC:                             110.1
Df Residuals:                      29   BIC:                             121.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.5691      0.064     